# Neo4j Interface

In [96]:
from neo4j import GraphDatabase
import re
import random
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import sys

class Neo4jDAO():
    def __init__(self, uri, user, pwd):
        super().__init__()
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)

    def close(self):
        if self.__driver is not None:
            self.__driver.close()

    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try:
            session = self.__driver.session(database=db) if db is not None else self.__driver.session()
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally:
            if session is not None:
                session.close()
        return response

    def createNode(self, objType, dictArgs):
        # figure out a good way to replace the strings in the dictionary
        the_str = f"merge (x:{objType} {dictArgs})"
        the_str = re.sub("'(\w+)':", r"\1:", the_str)
        return self.query(the_str)

    def createEdge(self, objTypeN1, argsN1, objTypeN2, argsN2, relType):
        the_str = f"match (x:{objTypeN1} {argsN1}) match (y:{objTypeN2} {argsN2}) merge (x)-[:{relType}]->(y)"
        the_str = re.sub("'(\w+)':", r"\1:", the_str)
        return self.query(the_str)

    def deleteNode(self, objType, dictArgs):
        the_str = f"match (x:{objType} {dictArgs}) detach delete x"
        the_str = re.sub("'(\w+)':", r"\1:", the_str)
        return self.query(the_str)

    def deleteEdge(self, objTypeN1, argsN1, objTypeN2, argsN2, relType):
        the_str = f"match (x:{objTypeN1} {argsN1})-[r:{relType}]-(y:{objTypeN2} {argsN2}) delete r"
        the_str = re.sub("'(\w+)':", r"\1:", the_str)
        return self.query(the_str)

    def updateNode(self, objType, dictArgs, update_name, update_data):
        the_str = f"match (x:{objType} {dictArgs}) set x.{update_name} = {update_data} return x"
        the_str = re.sub("'(\w+)':", r"\1:", the_str)
        return self.query(the_str)

    def updateEdge(self, objTypeN1, argsN1, objTypeN2, argsN2, relType, update_name, update_data):
        the_str = f"match (x:{objTypeN1} {argsN1})-[r:{relType}]-(y:{objTypeN2} {argsN2}) set r.{update_name} = {update_data} return r"
        the_str = re.sub("'(\w+)':", r"\1:", the_str)
        return self.query(the_str)

class Neo_Node:
    def __init__(self, name, properties):
        self.name = name
        self.properties = properties
        self.type = None

    def set_name(self, name):
        self.name = name

    def set_type(self, type):
        self.type = type

    def set_properties(self, properties):
        self.properties = properties

    def get_name(self):
        return self.name
        
    def get_properties(self):
        return self.properties
    
    def get_type(self):
        return self.type

In [130]:
def extract_tuples(record):
    tup_vals = []
    for i, val in enumerate(record):
        if i == 1:
            relationship_type = val.type
            relationship_type = relationship_type.replace('_', " ")
            tup_vals.append(relationship_type)
        else:
            node_name = None
            for name in val.labels:
                node_name = name
            node_properties = dict(val.items())
            node = Neo_Node(node_name, node_properties)
            tup_vals.append(node)

    return tuple(val for val in tup_vals)

def extract_facts(res):
    all_facts = []
    for record in res:
        new_tup = extract_tuples(record)
        all_facts.append(new_tup)

    return all_facts

# def verbalize_tuple_2(tuple_in):
#     string = tuple_in[0].get_properties()['name'] + " " + tuple_in[1] + " " + tuple_in[2].get_properties()['name']
#     # string = tuple_in[0].get_properties() + " " + tuple_in[1] + " " + tuple_in[2].get_properties()
#     return string

def verbalize_tuple_2(tuple_in):
    node1_props = ", ".join([f"{key}: {value}" for key, value in tuple_in[0].get_properties().items()])
    node2_props = ", ".join([f"{key}: {value}" for key, value in tuple_in[2].get_properties().items()])
    string = f"{node1_props} {tuple_in[1]} {node2_props}"
    return string


def facts_to_strings(facts):
    fact_str = []
    for fact in facts:
        fact_str.append(verbalize_tuple_2(fact))
    return fact_str

def cosine_similarity(user_string, facts_list):  
        cosine_scores = []
        X = user_string.lower()
        for fact in facts_list:
            Y = fact.lower()
            X_list = word_tokenize(X)
            Y_list = word_tokenize(Y)
            sw = stopwords.words('english')
            l1 = []; l2 = []
            X_set = {w for w in X_list if not w in sw}
            Y_set = {w for w in Y_list if not w in sw}
            rvector = X_set.union(Y_set)
            for w in rvector:
                if w in X_set: 
                    l1.append(1) # create a vector
                else: 
                    l1.append(0)
                if w in Y_set: 
                    l2.append(1)
                else: 
                    l2.append(0)
            c = 0
            # cosine formula 
            for i in range(len(rvector)):
                c+= l1[i]*l2[i]
            cosine = c / float((sum(l1)*sum(l2))**0.5)
            cosine_scores.append(cosine)
        if max(cosine_scores) == 0:
            rand_idx = random.randrange(len(cosine_scores))
            return facts_list[rand_idx], rand_idx, cosine_scores
        return facts_list[cosine_scores.index(max(cosine_scores))], cosine_scores.index(max(cosine_scores)), cosine_scores

def find_connections(dao, match_tuple):
        node_name = match_tuple[2].get_properties()['name']
        # query_str = f"MATCH (n)-[r]->(m) WHERE n.name = '{node_name}' RETURN n, r, m"
        query_str = "match (n)-[r]->(m) return n,r,m"
        res = dao.query(query_str)
        # Get turn neo4j result into fact tuples in a list
        all_tuples = extract_facts(res)
        print(sys.getsizeof(all_tuples))
        # Save and return all relevant information in string
        all_relevant_information = []
        all_relevant_information.append(verbalize_tuple_2(match_tuple))
        possible_tuples = []
        for tup in all_tuples:
            verbalize_tuple_2(tup)
            # iterate over each node
            for node in tup:
                # Skip relationship strings
                if type(node) == str:
                    continue
                # Find all outgoing connections from main quest node.
                if match_tuple[2].get_properties()['name'] == node.get_properties()['name']:
                    possible_tuples.append(tup)
        '''
        print("\nAll possible matches")
        for tup in possible_tuples:
            print(self.verbalize_tuple_2(tup))
        '''
        return possible_tuples

#Function to store all the relevent information as a list of dictionaries
def matched_coneections_all_details(possible_tuples):
    details_matched_relationships = []
    for relationship in possible_tuples:
        node1, relation, node2 = relationship
        details_matched_relationships.append({
            "node1": {
                "type": node1.get_type(),
                "properties": node1.get_properties()
            },
            "relation": relation,
            "node2": {
                "type": node2.get_type(),
                "properties": node2.get_properties()
            }
        })
    return details_matched_relationships
    


In [98]:
from dotenv import load_dotenv
import os
import nltk

load_dotenv()

url      = os.getenv("NEO4J_URI")
username = os.getenv("NEO4J_USERNAME")
password = os.getenv("NEO4J_PASSWORD")
database = os.getenv("NEO4J_DATABASE")

dao = Neo4jDAO(url, username, password)


In [124]:
query_str = "match (n)-[r]->(m) return n,r,m"
res = dao.query(query_str)

facts = extract_facts(res)

fact_strings = facts_to_strings(facts)
# print(fact_strings)
nltk.download('punkt')
nltk.download('stopwords')
player_input = "I want to A tier 2 melee weapon"
matched, idx, cosine_sim = cosine_similarity(player_input, fact_strings)

[nltk_data] Downloading package punkt to
[nltk_data]     /home/snehilaryan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/snehilaryan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [120]:
print(matched, idx)

tier: Tier 2, name: Errata, weight: 4.5, bonuses: ['+25% Armor Penetration', '+17% Burn Chance'], class: Melee LOCATED IN name: Kabuki 40


In [121]:
x = find_connections(dao, facts[idx])
print(x)

920
[(<__main__.Neo_Node object at 0x7f037f5a1450>, 'LOCATED IN', <__main__.Neo_Node object at 0x7f037f5a3580>), (<__main__.Neo_Node object at 0x7f037e74ee30>, 'LOCATED IN', <__main__.Neo_Node object at 0x7f037e74fb50>), (<__main__.Neo_Node object at 0x7f037e74dd50>, 'LOCATED IN', <__main__.Neo_Node object at 0x7f037e74d990>)]


In [141]:
y = str(matched_coneections_all_details(x))

In [140]:
print(str(y))


[{'node1': {'type': None, 'properties': {'name': 'Regina Jones'}}, 'relation': 'LOCATED IN', 'node2': {'type': None, 'properties': {'name': 'Kabuki'}}}, {'node1': {'type': None, 'properties': {'tier': 'Tier 2', 'name': 'Errata', 'weight': '4.5', 'bonuses': ['+25% Armor Penetration', '+17% Burn Chance'], 'class': 'Melee'}}, 'relation': 'LOCATED IN', 'node2': {'type': None, 'properties': {'name': 'Kabuki'}}}, {'node1': {'type': None, 'properties': {'name': 'Kabuki'}}, 'relation': 'LOCATED IN', 'node2': {'type': None, 'properties': {'name': 'Watson'}}}]


# Structured Content Creation using langchain

In [146]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

class QuestContent(BaseModel):
    title: str = Field(description="The title of the quest")
    npc_dialogue: str = Field(description="The dialogue of the NPC that gives the quest to the player")
    objective: str = Field(description="The objective of the quest")
    reward : str = Field(description="The reward for completing the quest")

temperature = 0.0
model = "gpt-4"
model = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0, response_format={ "type": "json_object" })
# structured_llm = model.with_structured_output(QuestContent)

model.invoke(
    "Generate a GTA V quest , respond in JSON with `title`, `npc_dialogue`, `objective` and `reward`",
)




/home/snehilaryan/game-llm-knowledge/rpg-with-knowledge-graphs/myenv/lib/python3.10/site-packages/langchain_openai/chat_models/base.py:299: UserWarning: WARNING! response_format is not default parameter.
                    response_format was transferred to model_kwargs.
                    Please confirm that response_format is what you intended.
  warnings.warn(


AIMessage(content='{\n    "title": "The Big Heist",\n    "npc_dialogue": "Hey there, I\'ve got a big job for you. We\'re going to rob the biggest bank in Los Santos. Are you in?",\n    "objective": "Gather a team of skilled criminals, plan the heist, execute the plan and escape with the money.",\n    "reward": "A cut of the stolen money and a new safehouse in a luxurious location."\n}')

In [147]:
from langchain_core.prompts import ChatPromptTemplate

prompt_context = '''
You are a rpg game quest generator system. Given some information and the type of quest which has been 
extracted from a knowledge graph you are supposed to generate the quest for the player. 
With the quest you are also expected to generate an acompynying dialouge that will be spoken 
by an NPC that will give the quest to the player. Also generate a reward as well as an exact 
objective for the quest. Respond in JSON with `title`, `npc_dialogue`, `objective` and `reward`
'''

template = ChatPromptTemplate.from_messages([
    ("system", prompt_context),
    ("human", "The player input is {player_prompt}, the quest type is {quest_type_ip}. The knowledge graph output is {kg_op}")
])

chain = template | model

prompt_value = chain.invoke(
    {
        "kg_op": y,
        "player_prompt": player_input,
        "quest_type_ip": "combat"
    }
)
print(prompt_value)



content='{\n    "title": "The Errata Blade",\n    "npc_dialogue": "Ah, greetings traveler! I am Regina Jones, the local blacksmith here in Kabuki. I have heard of a powerful Tier 2 melee weapon called Errata that is said to be located in the district of Watson. Bring me this weapon and I shall reward you handsomely.",\n    "objective": "Travel to Watson and retrieve the Errata blade from its hiding place.",\n    "reward": "A Tier 2 melee weapon - Errata (Stats: +25% Armor Penetration, +17% Burn Chance)"\n}'


In [104]:
x[0][0].get_properties()

{'name': 'Mitch'}

In [105]:
x[1][2].get_properties()

{'name': 'Junkyard'}

In [106]:
for relationship in x:
    node1, relation, node2 = relationship
    print(f"Node 1 properties: {node1.get_properties()}")
    print(f"Relation: {relation}")
    print(f"Node 2 properties: {node2.get_properties()}")
    print("---")

Node 1 properties: {'name': 'Mitch'}
Relation: LOCATED IN
Node 2 properties: {'name': 'Junkyard'}
---
Node 1 properties: {'tier': 'Tier 5', 'name': 'BREAKTHROUGH', 'weight': '11.7', 'bonuses': ['+300% Headshot Damage Multiplier', '+100% Armor Penetration', '1.25sec Charge Time'], 'type': 'Sniper Rifle'}
Relation: LOCATED IN
Node 2 properties: {'name': 'Junkyard'}
---
Node 1 properties: {'name': 'Junkyard'}
Relation: LOCATED IN
Node 2 properties: {'name': 'Outskirts'}
---


In [126]:
matched_relationships = []
for relationship in x:
    node1, relation, node2 = relationship
    matched_relationships.append({
        "node1": {
            "type": node1.get_type(),
            "properties": node1.get_properties()
        },
        "relation": relation,
        "node2": {
            "type": node2.get_type(),
            "properties": node2.get_properties()
        }
    })

print(matched_relationships)

[{'node1': {'type': None, 'properties': {'name': 'Regina Jones'}}, 'relation': 'LOCATED IN', 'node2': {'type': None, 'properties': {'name': 'Kabuki'}}}, {'node1': {'type': None, 'properties': {'tier': 'Tier 2', 'name': 'Errata', 'weight': '4.5', 'bonuses': ['+25% Armor Penetration', '+17% Burn Chance'], 'class': 'Melee'}}, 'relation': 'LOCATED IN', 'node2': {'type': None, 'properties': {'name': 'Kabuki'}}}, {'node1': {'type': None, 'properties': {'name': 'Kabuki'}}, 'relation': 'LOCATED IN', 'node2': {'type': None, 'properties': {'name': 'Watson'}}}]


In [107]:
output = ""
for relationship in x:
    node1, relation, node2 = relationship
    output += f"Node 1 properties: {node1.get_properties()}\n"
    output += f"Relation: {relation}\n"
    output += f"Node 2 properties: {node2.get_properties()}\n"
    output += "---\n"

# QuestEngine

In [108]:
gathering_start_words = ['Find', 'Gather', 'Collect', 'Retrieve', 'Get me', 'Bring back', 'Obtain', 'Get']
exploration_start_words = ['Go to', 'Visit', 'Go see', 'Travel to', 'Journey to', ' Explore']
combat_start_words = ['Fight', 'Slay', 'Kill', 'Defeat', 'Vanquish', 'Eliminate']
possible_relationships = ['located_in', 'has', 'protected_by', 'wants_killed']


class Quest:
    def __init__(self, quest_type, start, quest_target, location, enemy, person_to_visit, item_to_retrieve, number_to_collect=None, number_to_defeat=None):
        self.quest_type = quest_type
        self.start = start
        self.quest_target = quest_target
        self.location = location
        self.enemy = enemy
        self.person_to_visit = person_to_visit
        self.item_to_retrieve = item_to_retrieve
        self.number_to_collect = number_to_collect
        self.number_to_defeat = number_to_defeat

    def generate_description(self):
        if self.quest_type == "Gathering":
            return f"{self.start} {self.number_to_collect} {self.quest_target} {self.location} protected by {self.enemy}"
        elif self.quest_type == "Exploration":
            return f"{self.start} {self.quest_target} and meet {self.person_to_visit}"
        elif self.quest_type == "Combat":
            return f"{self.start} {self.number_to_defeat} {self.quest_target} located in {self.location} and retrieve {self.item_to_retrieve}"
        else:
            return "Invalid quest type"

# Example usage:
quest = Quest("Gathering", "Start at dawn,", "10 apples", "the Forbidden Forest", "a troll", None, None, 10)
print(quest.generate_description())

Start at dawn, 10 10 apples the Forbidden Forest protected by a troll


## Using LLM to generate quest, dialouge, reward and objective

In [109]:
from openai import OpenAI
client = OpenAI()

prompt = '''
you are a  rpg game quest generator system. Given some information and the type of quest which has been 
extracted from a knowledge graph you are supposed to generate the quest for the player. With the quest you are 
also expected to generate an acompynying dialouge that will be spoken 
by an NPC that will give the quest to the player. Also generate a reward as well as an exact objective for the quest
'''
quest_type = "Combat"
player_input = "I want to go on a quest with Jackie Welles"

response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": f"{prompt} the knowledge graph output is {output} and the player input is {player_input} and the quest type is {quest_type}"}
    # {"role": "assistant", "content": output},
    # {"role": "user", "content": player_input},
    # {"role": "assistant", "content": quest_type}
  ]
)



In [110]:
response.choices[0].message.content

'Quest: Sniper\'s Delight\n\nNPC Dialogue: \nAh, greetings adventurer! My name is Mitch, the local weapon smith around these parts. I have a special task for a skilled warrior like yourself. You see, a powerful sniper rifle known as "BREAKTHROUGH" has been spotted in the Junkyard, located in the Outskirts. Rumor has it that this rifle comes with incredible bonuses such as a +300% Headshot Damage Multiplier, +100% Armor Penetration, and a 1.25sec Charge Time. I believe this weapon would be perfect for someone of your expertise in combat. Retrieve the "BREAKTHROUGH" rifle for me, and you shall be greatly rewarded.\n\nObjective: \nHead to the Junkyard in the Outskirts and retrieve the "BREAKTHROUGH" sniper rifle.\n\nReward: \n- Exclusive access to purchase rare sniper rifle modifications\n- Gold coins\n- Experience points\n\nGood luck on your quest, brave adventurer!'

In [111]:
response.choices[0].message.content

'Quest: Sniper\'s Delight\n\nNPC Dialogue: \nAh, greetings adventurer! My name is Mitch, the local weapon smith around these parts. I have a special task for a skilled warrior like yourself. You see, a powerful sniper rifle known as "BREAKTHROUGH" has been spotted in the Junkyard, located in the Outskirts. Rumor has it that this rifle comes with incredible bonuses such as a +300% Headshot Damage Multiplier, +100% Armor Penetration, and a 1.25sec Charge Time. I believe this weapon would be perfect for someone of your expertise in combat. Retrieve the "BREAKTHROUGH" rifle for me, and you shall be greatly rewarded.\n\nObjective: \nHead to the Junkyard in the Outskirts and retrieve the "BREAKTHROUGH" sniper rifle.\n\nReward: \n- Exclusive access to purchase rare sniper rifle modifications\n- Gold coins\n- Experience points\n\nGood luck on your quest, brave adventurer!'